## GANITE(Tensorflow): Train and Evaluation

This notebook presents the solution for training and evaluating GANITE(Tensoflow version).

The implementation of GANITE is adapted in the local `ite` library.

First, make sure that all the depends are installed.
```
pip install -r requirements.txt
pip install .
```

### Setup

First, we import all the dependencies necessary for the task.

In [ ]:
# Double check that we are using the correct interpreter.
import sys
print(sys.executable)

# Disable TF logging
import os 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
    
# Import
import ite.algs.ganite.model as alg
import ite.datasets as ds
import ite.utils.tensorflow as utils

from matplotlib import pyplot as plt

import tensorflow.compat.v1 as tf

### Load the Dataset



In [ ]:
train_ratio = 0.8
 
dataloader = ds.load("twins", train_ratio)
[Train_X, Train_T, Train_Y, Opt_Train_Y, Test_X, Test_Y] = dataloader

### Load the model

In [ ]:
dim = len(Train_X[0])
dim_hidden = 8
dim_outcome = Test_Y.shape[1]
 
model = alg.Ganite(
    dim, # number of features
    dim_hidden, # size of the hidden layers
    dim_outcome, # size of the output
    num_iterations=10000, # number of training iterations
    alpha=2, # alpha hyperparameter, used for the Generator block loss
    beta=2, # beta hyperparameter, used for the ITE block loss
    minibatch_size=128, # data batch size
    num_discr_iterations=10, # number of iterations executed by the discriminator.
)

assert model is not None

### Train

In [ ]:
metrics = model.train(*dataloader)

### Plot train metrics

In [ ]:
metrics.plot(plt, thresholds = [0.2, 0.25, 0.3, 0.35])

metrics.print()

### Predict

You can use run inferences on the model and evaluate the output.

In [ ]:
sess = tf.InteractiveSession()

hat_y = model.predict(Test_X)

utils.sqrt_PEHE(hat_y, Test_Y).eval()

### Test
Will can run inferences and get metrics directly

In [ ]:
test_metrics = model.test(Test_X, Test_Y)

test_metrics.print()